In [1]:
# Importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

import pandasql as psql

In [2]:
# load the Auto Quote ins data

AutoIns = pd.read_csv(r"D:\iiit notes\Internship\31 season 25th jun-2021\Auto_Quote_Data_V2.0.csv", header=0)
AutoIns.head()

,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Sal_Range1,Sal_Range2,Coverage,Veh_Usage,Annual_Miles_Range,Vehicl_Cost_Range1,Vehicl_Cost_Range2,Re_Quote,Quoted_Premium,Policy_Bind
0,AQ-C-139212,EA,2020/04/25,2020/06/23,2020/05/23,C,2156,Car,3,3,...,> $ 25 K <= $ 40 K,> 25 K <= 40 K,Balanced,Commute,> 55 K,> $ 10 K <= $ 20 K,> 10 K <= 20 K,No,693.86,Yes
1,AQ-F-136117,EA,2020/02/21,2020/04/20,NaN,F,2153,Van,2,2,...,> $ 40 K <= $ 60 K,> 40 K <= 60 K,Balanced,Pleasure,> 7.5 K & <= 15 K,<= $ 10 K,<= 10 K,No,635.96,No
2,AQ-F-126801,EA,2020/06/19,2020/08/17,2020/07/12,F,2056,Truck,2,1,...,> $ 40 K <= $ 60 K,> 40 K <= 60 K,Basic,Commute,> 35 K & <= 45 K,> $ 10 K <= $ 20 K,> 10 K <= 20 K,No,780.64,Yes
3,AQ-E-143467,EA,2020/05/02,2020/06/30,2020/05/24,E,2138,Car,1,2,...,> $ 90 K,> 90 K,Basic,Pleasure,<= 7.5 K,<= $ 10 K,<= 10 K,No,723.15,Yes
4,AQ-C-143827,EA,2020/02/12,2020/04/11,2020/02/25,C,2327,Truck,3,1,...,<= $ 25 K,<= 25 K,Basic,Pleasure,> 35 K & <= 45 K,<= $ 10 K,<= 10 K,No,738.14,Yes


In [3]:
# Convert the target varaible data type into integer
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('No', '0')
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('Yes', '1')
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].astype(int)

In [4]:
# Count the target or dependent varaible by '0' or '1' and
# their proportion (> 10 : 1, then the dataset is imbalance dataset)

Policy_Bind_count = AutoIns.Policy_Bind.value_counts()
print('Class 0:', Policy_Bind_count[0])
print('Class 1:', Policy_Bind_count[1])
print('Proportion:', round(Policy_Bind_count[0]/ Policy_Bind_count[1], 2), ': 1')
print('Total Records:', len(AutoIns))

Class 0: 113757
Class 1: 32502
Proportion: 3.5 : 1
Total Records: 146259


In [5]:
AutoIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Quote_Num           146259 non-null  object 
 1   Agent_Type          146259 non-null  object 
 2   Q_Creation_DT       146259 non-null  object 
 3   Q_Valid_DT          146259 non-null  object 
 4   Policy_Bind_DT      32502 non-null   object 
 5   Region              146259 non-null  object 
 6   Agent_Num           146259 non-null  int64  
 7   Policy_Type         146259 non-null  object 
 8   HH_Vehicles         146259 non-null  int64  
 9   HH_Drivers          146259 non-null  int64  
 10  Driver_Age          146259 non-null  int64  
 11  Driving_Exp         146259 non-null  int64  
 12  Prev_Accidents      146259 non-null  int64  
 13  Prev_Citations      146259 non-null  int64  
 14  Gender              146259 non-null  object 
 15  Marital_Status      146259 non-nul

In [6]:
# Ranges and new column as 'QP_Range' from 'Quoted_premium'

AutoIns['QP_Range'] = pd.cut(AutoIns['Quoted_Premium'], [0, 800, 1000, 1200, 9999],
                             labels = ['0-800', '801-1000', '1001-1200', '>1200'])

In [7]:
# Dropping the varaibles which are not impacting the target variable

AutoIns = AutoIns.drop(['Quote_Num', 'Agent_Num', 'Q_Creation_DT',
                        'Q_Valid_DT', 'Policy_Bind_DT', 'Sal_Range1',
                        'Vehicl_Cost_Range1', 'Quoted_Premium'], axis=1)

In [8]:
AutoIns = pd.get_dummies(AutoIns, columns=['Agent_Type','Region', 'Policy_Type', 'Gender', 'Marital_Status', 'Education',
                                          'Sal_Range2', 'Coverage', 'Veh_Usage', 'Annual_Miles_Range', 'Vehicl_Cost_Range2',
                                          'Re_Quote', 'QP_Range'])

In [9]:
AutoIns.head()

,HH_Vehicles,HH_Drivers,Driver_Age,Driving_Exp,Prev_Accidents,Prev_Citations,Policy_Bind,Agent_Type_EA,Agent_Type_IA,Region_A,...,Vehicl_Cost_Range2_> 10 K <= 20 K,Vehicl_Cost_Range2_> 20 K <= 30 K,Vehicl_Cost_Range2_> 30 K <= 40 K,Vehicl_Cost_Range2_> 40 K,Re_Quote_No,Re_Quote_Yes,QP_Range_0-800,QP_Range_801-1000,QP_Range_1001-1200,QP_Range_>1200
0,3,3,43,26,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,0
1,2,2,30,13,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,1,25,8,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,0
3,1,2,43,26,0,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0
4,3,1,40,23,0,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,0


In [10]:
# Identify the dependent and Target variables

IndepVar =[]
for col in AutoIns.columns:
    if col != 'Policy_Bind':
        IndepVar.append(col)
        
TargetVar = 'Policy_Bind'
x = AutoIns[IndepVar]
y = AutoIns[TargetVar]

In [11]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

In [12]:
AutoIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 60 columns):
 #   Column                                Non-Null Count   Dtype
---  ------                                --------------   -----
 0   HH_Vehicles                           146259 non-null  int64
 1   HH_Drivers                            146259 non-null  int64
 2   Driver_Age                            146259 non-null  int64
 3   Driving_Exp                           146259 non-null  int64
 4   Prev_Accidents                        146259 non-null  int64
 5   Prev_Citations                        146259 non-null  int64
 6   Policy_Bind                           146259 non-null  int32
 7   Agent_Type_EA                         146259 non-null  uint8
 8   Agent_Type_IA                         146259 non-null  uint8
 9   Region_A                              146259 non-null  uint8
 10  Region_B                              146259 non-null  uint8
 11  Region_C                  

# DECISION TREE

# Decision Tree with random sampling

In [13]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as 'Scaling'

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train['Driver_Age'] = sc.fit_transform(x_train['Driver_Age'].values.reshape(-1, 1))
x_train['Driving_Exp'] = sc.fit_transform(x_train['Driving_Exp'].values.reshape(-1, 1))
x_test['Driver_Age'] = sc.fit_transform(x_test['Driver_Age'].values.reshape(-1, 1)) 
x_test['Driving_Exp'] = sc.fit_transform(x_test['Driving_Exp'].values.reshape(-1, 1)) 
# Convert to dataframe x_train = pd.DataFrame(x_train) x_test = pd.DataFrame(x_test)

In [14]:
# Build the decision tree model with random sampling 
from sklearn.tree import DecisionTreeClassifier 
AutoInsDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, 
                                   max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                   min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
                                   min_weight_fraction_leaf=0.0, random_state=None, splitter='best') 
AutoInsDT = AutoInsDT.fit(x_train,y_train)

In [15]:
# Predict the model with test data set 

y_pred = AutoInsDT.predict(x_test)

In [16]:
# Display confusion matrix and classifiction report 
from sklearn.metrics import classification_report, confusion_matrix 

print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test, y_pred))

[[25378  8684]
 [ 7366  2450]]
              precision    recall  f1-score   support

           0       0.78      0.75      0.76     34062
           1       0.22      0.25      0.23      9816

    accuracy                           0.63     43878
   macro avg       0.50      0.50      0.50     43878
weighted avg       0.65      0.63      0.64     43878



In [17]:
# Evaluate the model performance by metrics 
from sklearn import metrics 
from sklearn.metrics import roc_curve, roc_auc_score 

# Model Accuracy: how often is the classifier correct? 
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%") 

# Model Precision: what percentage of positive tuples are labeled as such? 
print("Precision:", (round(metrics.precision_score(y_test, y_pred) * 100, 2)), '%') 

# Model Recall: what percentage of positive tuples are labelled as such? 
print("Recall:", (round(metrics.recall_score(y_test, y_pred) * 100, 2)), "%") 

# Model f1-score: weighted average of Precision & Recall 
print("f1-score:", (round(metrics.f1_score(y_test, y_pred) * 100, 2)), '%') 

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

Accuracy: 63.42 %
Precision: 22.0 %
Recall: 24.96 %
f1-score: 23.39 %
roc_auc_score: 0.497


# Decision tree over sampling

In [18]:
# Random oversampling can be implemented using the RandomOverSampler class

from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
x_over, y_over = oversample.fit_resample(x, y)
print(x_over.shape)
print(y_over.shape)

(227514, 59)
(227514,)


In [19]:
x.shape

(146259, 59)

In [20]:
# To know the count of major and minor classes

y_over.value_counts()

0    113757
1    113757
Name: Policy_Bind, dtype: int64

In [21]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xo_train, xo_test, yo_train, yo_test = train_test_split(x_over, y_over, test_size = 0.3, random_state=42)

In [22]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#x_train = sc.fit_transform(x_train)
#x_test = sc.fit_transform(x_test)

xo_train['Driver_Age'] = sc.fit_transform(xo_train['Driver_Age'].values.reshape(-1, 1))
xo_train['Driving_Exp'] = sc.fit_transform(xo_train['Driving_Exp'].values.reshape(-1, 1))
xo_test['Driver_Age'] = sc.fit_transform(xo_test['Driver_Age'].values.reshape(-1, 1))
xo_test['Driving_Exp'] = sc.fit_transform(xo_test['Driving_Exp'].values.reshape(-1, 1))

# convert to dataframe

xo_train = pd.DataFrame(xo_train)
xo_test = pd.DataFrame(xo_test)

In [23]:
# To build the decision tree model with Under Sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_O = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                      max_depth=None, max_features=None, max_leaf_nodes=None,
                                      min_impurity_decrease=0.0, min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                      random_state=None, splitter='best')

AutoInsDT_O = AutoInsDT_O.fit(xo_train, yo_train)

In [24]:
# Predict with test data

yo_pred = AutoInsDT_O.predict(xo_test)

In [25]:
# Display confusion matrix and classifiction report 

from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

[[23984 10093]
 [ 2052 32126]]
              precision    recall  f1-score   support

           0       0.92      0.70      0.80     34077
           1       0.76      0.94      0.84     34178

    accuracy                           0.82     68255
   macro avg       0.84      0.82      0.82     68255
weighted avg       0.84      0.82      0.82     68255



In [26]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

Accuracy: 82.21 %
Precision: 76.09 %
Recall: 94.0 %
f1-score: 84.1 %
roc_auc_score: 0.822


# Combining Under & Oversampling - Decision Tree

In [27]:
# Combining Random Oversampling and Undersampling 

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# define oversampling strategy

over = RandomOverSampler(sampling_strategy=0.70)

# fit and apply the transform

x2, y2 = over.fit_resample(x,y)
print(x2.shape)
print(y2.shape)

# define undersampling strategy

under = RandomUnderSampler(sampling_strategy=0.70)

# fit and apply the transform

x3, y3 = under.fit_resample(x2, y2)

print(x3.shape)
print(y3.shape)


(193386, 59)
(193386,)
(193384, 59)
(193384,)


In [28]:
y3.value_counts()

0    113755
1     79629
Name: Policy_Bind, dtype: int64

In [29]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xc_train, xc_test, yc_train, yc_test = train_test_split(x3, y3, test_size = 0.30, random_state = 42)


In [30]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xc_train['Driver_Age'] = sc.fit_transform(xc_train['Driver_Age'].values.reshape(-1, 1))
xc_train['Driving_Exp'] = sc.fit_transform(xc_train['Driving_Exp'].values.reshape(-1, 1))
xc_test['Driver_Age'] = sc.fit_transform(xc_test['Driver_Age'].values.reshape(-1, 1))
xc_test['Driving_Exp'] = sc.fit_transform(xc_test['Driving_Exp'].values.reshape(-1, 1))

# convert to dataframe

xc_train = pd.DataFrame(xc_train)
xc_test = pd.DataFrame(xc_test)


In [31]:
# To build the decision tree model with Under sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_C = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                     max_depth=None, max_features=None, max_leaf_nodes=None,
                                     min_impurity_decrease=0.0, min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                     random_state=None, splitter='best')

AutoInsDT_C = AutoInsDT_C.fit(xc_train,yc_train)

In [32]:
# Predict with test data

yc_pred = AutoInsDT_C.predict(xc_test)

In [33]:
# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

[[24504  9500]
 [ 3568 20444]]
              precision    recall  f1-score   support

           0       0.87      0.72      0.79     34004
           1       0.68      0.85      0.76     24012

    accuracy                           0.77     58016
   macro avg       0.78      0.79      0.77     58016
weighted avg       0.79      0.77      0.78     58016



In [34]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

Accuracy: 77.48 %
Precision: 68.27 %
Recall: 85.14 %
f1-score: 75.78 %
roc_auc_score: 0.786


# Under sampling -Decision tree

In [35]:
# undersampling 

from imblearn.under_sampling import RandomUnderSampler

under = RandomUnderSampler(sampling_strategy = 'majority')
x_under, y_under = under.fit_resample(x, y)

print(x_under.shape)
print(y_under.shape)


(65004, 59)
(65004,)


In [36]:
y_under.value_counts()

0    32502
1    32502
Name: Policy_Bind, dtype: int64

In [37]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

xu_train, xu_test, yu_train, yu_test = train_test_split(x_under, y_under, test_size=0.3, random_state=42)

In [38]:
# Feature Scaling - Each independent variable is in different range. The process of transforming all the
# features in the given data set to a fixed range is known as ‘Scaling’

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xu_train['Driver_Age'] = sc.fit_transform(xu_train['Driver_Age'].values.reshape(-1,1))
xu_train['Driving_Exp'] = sc.fit_transform(xu_train['Driving_Exp'].values.reshape(-1,1))
xu_train['Driver_Age'] = sc.fit_transform(xu_train['Driver_Age'].values.reshape(-1,1))
xu_train['Driving_Exp'] = sc.fit_transform(xu_train['Driving_Exp'].values.reshape(-1,1))

# convert to dataframe

xu_train =  pd.DataFrame(xu_train)
xu_test = pd.DataFrame(xu_test)

In [39]:
# To build the decision tree model with Under sampling

from sklearn.tree import DecisionTreeClassifier

AutoInsDT_C = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, 
                                     min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                                     random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                     min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

AutoInsDT_C = AutoInsDT_C.fit(xu_train, yu_train)

In [40]:
# Predict with test data

yu_pred = AutoInsDT_C.predict(xu_test)

In [41]:
# Display confusion matrix and classification report 

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

[[4511 5242]
 [4506 5243]]
              precision    recall  f1-score   support

           0       0.50      0.46      0.48      9753
           1       0.50      0.54      0.52      9749

    accuracy                           0.50     19502
   macro avg       0.50      0.50      0.50     19502
weighted avg       0.50      0.50      0.50     19502



In [42]:
# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

Accuracy: 77.48 %
Precision: 68.27 %
Recall: 85.14 %
f1-score: 75.78 %
roc_auc_score: 0.786


# RANDOM FOREST

# Randomforest with oversampling

In [43]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_O = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_O = AutoInsRF_O.fit(xo_train, yo_train)


# Predict the model with test data set

yo_pred = AutoInsRF_O.predict(xo_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

[[33740   337]
 [ 2904 31274]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     34077
           1       0.99      0.92      0.95     34178

    accuracy                           0.95     68255
   macro avg       0.96      0.95      0.95     68255
weighted avg       0.96      0.95      0.95     68255

Accuracy: 95.25 %
Precision: 95.25 %
Recall: 95.25 %
f1-score: 95.25 %
roc_auc_score: 0.953


# RandomForest with Combined Over sampling and Under sampling

In [44]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_C = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_C = AutoInsRF_C.fit(xc_train, yc_train)


# Predict the model with test data set

yc_pred = AutoInsRF_C.predict(xc_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

[[33857   147]
 [ 5069 18943]]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93     34004
           1       0.99      0.79      0.88     24012

    accuracy                           0.91     58016
   macro avg       0.93      0.89      0.90     58016
weighted avg       0.92      0.91      0.91     58016

Accuracy: 91.01 %
Precision: 91.01 %
Recall: 91.01 %
f1-score: 91.01 %
roc_auc_score: 0.892


# RandomForest with Under sampling

In [45]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_C = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_C = AutoInsRF_C.fit(xu_train, yu_train)


# Predict the model with test data set

yu_pred = AutoInsRF_C.predict(xu_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yu_test, yu_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yu_test, yu_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yu_test, yu_pred), 3)) 

[[3207 6546]
 [3204 6545]]
              precision    recall  f1-score   support

           0       0.50      0.33      0.40      9753
           1       0.50      0.67      0.57      9749

    accuracy                           0.50     19502
   macro avg       0.50      0.50      0.48     19502
weighted avg       0.50      0.50      0.48     19502

Accuracy: 50.01 %
Precision: 50.01 %
Recall: 50.01 %
f1-score: 50.01 %
roc_auc_score: 0.5


# RandomForest with randam sampling

In [46]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

AutoInsRF_C = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='gini', max_depth=None, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)
AutoInsRF_C = AutoInsRF_C.fit(x_train, y_train)


# Predict the model with test data set
    
y_pred = AutoInsRF_C.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Evaluate the model performance by mtrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

[[34051    11]
 [ 9812     4]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34062
           1       0.27      0.00      0.00      9816

    accuracy                           0.78     43878
   macro avg       0.52      0.50      0.44     43878
weighted avg       0.66      0.78      0.68     43878

Accuracy: 77.61 %
Precision: 77.61 %
Recall: 77.61 %
f1-score: 77.61 %
roc_auc_score: 0.5


# LOGISTIC REGRESSION

# Oversampling - Logestic regression

In [48]:
# To bulid the 'Logistic Regression' model with oversampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_O = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

AutoInsLR_O = AutoInsLR_O.fit(xo_train,yo_train)

# Predict the model with test data set

yo_pred = AutoInsLR_O.predict(xo_test)

# Display the confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yo_test, yo_pred))
print(classification_report(yo_test, yo_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yo_test, yo_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yo_test, yo_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yo_test, yo_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yo_test, yo_pred), 3))

[[16977 17100]
 [16729 17449]]
              precision    recall  f1-score   support

           0       0.50      0.50      0.50     34077
           1       0.51      0.51      0.51     34178

    accuracy                           0.50     68255
   macro avg       0.50      0.50      0.50     68255
weighted avg       0.50      0.50      0.50     68255

Accuracy: 50.44 %
Precision: 50.44 %
Recall: 50.44 %
f1-score: 50.44 %
roc_auc_score: 0.504


# Combining under - Over sampling Logistic regression

In [49]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_C = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
AutoInsLR_C = AutoInsLR_C.fit(xc_train,yc_train)

# Predict the model with test data set

yc_pred = AutoInsLR_C.predict(xc_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yc_test, yc_pred))
print(classification_report(yc_test, yc_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yc_test, yc_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yc_test, yc_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yc_test, yc_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yc_test, yc_pred), 3))

[[34004     0]
 [24012     0]]
              precision    recall  f1-score   support

           0       0.59      1.00      0.74     34004
           1       0.00      0.00      0.00     24012

    accuracy                           0.59     58016
   macro avg       0.29      0.50      0.37     58016
weighted avg       0.34      0.59      0.43     58016

Accuracy: 58.61 %
Precision: 58.61 %
Recall: 58.61 %
f1-score: 58.61 %
roc_auc_score: 0.5


# Under sampling with Logistic regression

In [52]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR_U = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

AutoInsLR_U = AutoInsLR_U.fit(xu_train,yu_train)

# Predict the model with test data set

yu_pred = AutoInsLR_U.predict(xu_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(yu_test, yu_pred))
print(classification_report(yu_test, yu_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(yu_test, yu_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(yu_test, yu_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(yu_test, yu_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(yu_test, yu_pred), 3))

[[  14 9739]
 [  18 9731]]
              precision    recall  f1-score   support

           0       0.44      0.00      0.00      9753
           1       0.50      1.00      0.67      9749

    accuracy                           0.50     19502
   macro avg       0.47      0.50      0.33     19502
weighted avg       0.47      0.50      0.33     19502

Accuracy: 49.97 %
Precision: 49.97 %
Recall: 49.97 %
f1-score: 49.97 %
roc_auc_score: 0.5


# Ramdom sampling with Logistic Regression

In [53]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

AutoInsLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
AutoInsLR = AutoInsLR.fit(x_train,y_train)

# Predict the model with test data set

y_pred = AutoInsLR.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred), 3))

[[34062     0]
 [ 9816     0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87     34062
           1       0.00      0.00      0.00      9816

    accuracy                           0.78     43878
   macro avg       0.39      0.50      0.44     43878
weighted avg       0.60      0.78      0.68     43878

Accuracy: 77.63 %
Precision: 77.63 %
Recall: 77.63 %
f1-score: 77.63 %
roc_auc_score: 0.5
